In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df_texts = pd.read_csv('../input/russian-language-toxic-comments/labeled.csv')

In [ ]:
import string

In [ ]:
from nltk.corpus import stopwords
import pymorphy2

In [ ]:
stop_words = stopwords.words('russian')
morph = pymorphy2.MorphAnalyzer()

In [ ]:
import re

In [ ]:
def clean_text(comment):
    comment = comment.lower()

    comment = re.sub(r"@[а-яА-ЯA-Za-z0-9]+", ' ', comment)

    comment = re.sub(r"https?://[A-Za-z0-9./]+", ' ', comment)
    comment = re.sub(r"http?://[A-Za-z0-9./]+", ' ', comment)
        
    comment = re.sub(r"[^а-яА-Яa-zA-Z.!?']", ' ', comment)

    comment = re.sub(r" +", ' ', comment)
    
    for ch in string.punctuation:
        comment = comment.replace(ch, ' ')
        
    return " ".join([morph.parse(word)[0].normal_form for word in comment.split() if not word in stop_words])

In [ ]:
df_texts['clean_comment'] = df_texts.comment.apply(clean_text)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
vectorizer = TfidfVectorizer(max_features=2000)
vectors = vectorizer.fit_transform(df_texts['clean_comment'])

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(vectors, df_texts['toxic'], test_size=0.3)

In [ ]:
model = RandomForestClassifier()
model.fit(X_train, y_train)

In [ ]:
f1_score(model.predict(X_test), y_test)

In [ ]:
try:
    %tensorflow_version 2.x
except Exception:
    pass
import tensorflow as tf

import tensorflow_hub as hub

from tensorflow.keras import layers
from tensorflow.keras.models import load_model
import bert
from transformers import AutoTokenizer

In [ ]:
from bert import tokenization

In [ ]:
FullTokenizer = tokenization.FullTokenizer
#FullTokenizer = AutoTokenizer.from_pretrained('bert-base-cased')
bert_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1",
                            trainable=False)
vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
do_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = FullTokenizer(vocab_file, do_lower_case)

In [ ]:
def encode_sentence(sent):
    return ["[CLS]"] + tokenizer.tokenize(sent)[:510] + ["[SEP]"]

In [ ]:
data_inputs = [encode_sentence(sentence) for sentence in df_texts['clean_comment']]

In [ ]:
tokens = set()
for text in data_inputs:
    for token in text[1:-1]:
        tokens.add(token)
tokens = sorted(tokens)